In [64]:
import folium
import pandas as pd 

# 簡單版本：使用預設的 OpenStreetMap tiles 先測試
m = folium.Map(
    location=(24.161259, 120.631192
),
    zoom_start=13,
    tiles="Cartodb dark_matter"
    
)

# 添加一個標記以確認地圖能正常工作
folium.Marker(
    location=(24.161259, 120.631192),
    popup='site',
    tooltip='site'
).add_to(m)


In [65]:
m
#m.save("index2.html")

In [66]:
import pandas as pd
from pathlib import Path

# 建立安全的路徑引用（使用相對路徑）
p = Path('data/merged_data/MRT.json')
print('Path ->', p.resolve())
print('Exists ->', p.exists())

if p.exists():
    try:
        # 嘗試以一般 JSON 格式讀取
        df_mrtroute = pd.read_json(p)
    except ValueError:
        # 若為每行一個 JSON（ndjson），改用 lines=True
        df_mrtroute = pd.read_json(p, lines=True)
    print('Loaded rows,cols ->', df_mrtroute.shape)
    display(df_mrtroute.head())
else:
    raise FileNotFoundError(f'MRT.json not found at: {p}')


Path -> C:\DCCG_final project\final-project\data\merged_data\MRT.json
Exists -> True
Loaded rows,cols -> (18, 3)


,欄位名稱,數值,經緯度
0,北屯總站,21818,"24.189132,120.70864"
1,舊社,30414,"24.182289,120.707294"
2,松竹,80013,"24.180807,120.70145"
3,四維國小,53208,"24.171243,120.693295"
4,文心崇德,91029,"24.172194,120.684869"


In [67]:
# 將 MRT 站點加入地圖，並根據「數值」調整點的大小
import pandas as pd
import numpy as np

# 確認 df_mrtroute 存在
if 'df_mrtroute' not in globals():
    raise NameError('df_mrtroute 未定義，請先執行載入 MRT 的 cell')

# 解析經緯度欄位 (格式: 'lat,lon')
coords = df_mrtroute['經緯度'].astype(str).str.split(',', expand=True)
df_mrtroute['lat'] = coords[0].astype(float)
df_mrtroute['lon'] = coords[1].astype(float)

# 取出數值欄並轉為數字
vals = pd.to_numeric(df_mrtroute['數值'], errors='coerce').fillna(0)
min_val, max_val = vals.min(), vals.max()

# 定義映射函數 -> 半徑 pixels
min_radius, max_radius = 4, 18

def scale_radius(v):
    if pd.isna(v):
        v = 0
    if max_val == min_val:
        return (min_radius + max_radius) / 2
    return min_radius + (float(v) - min_val) / (max_val - min_val) * (max_radius - min_radius)

for _, r in df_mrtroute.iterrows():
    radius = scale_radius(r['數值'])
    folium.CircleMarker(
        location=(r['lat'], r['lon']),
        radius=radius,
        color='lightgreen',
        fill=True,
        fill_color='lightgreen',
        fill_opacity=0.6,
        popup=f"{r['欄位名稱']}<br>數值: {r['數值']}",
        tooltip=r['欄位名稱']
    ).add_to(m)

# 加上圖層控制，顯示並儲存
folium.LayerControl().add_to(m)

m




In [68]:
m.save('map_with_mrt.html')
print('Saved map_with_mrt.html')

Saved map_with_mrt.html
